<a href="https://colab.research.google.com/github/MIcoder003/Tourist-Recommendation-System/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Data

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse
from scipy.spatial.distance import correlation

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Sem 7/RecommendeSystems/tokyo_clean.csv")

In [ ]:
# arrannge according to userIds so that similar users are continious

df1 = df1.sort_values(by='user_id')

In [ ]:
# Assigning user Ids

import pandas as pd

def replace_user_ids_with_numbers(df):
    # Create a dictionary to map unique user IDs to numbers
    user_id_mapping = {}
    current_number = 1  # Start numbering from 1

    # Iterate through the rows of the DataFrame
    for index, row in df.iterrows():
        user_id = row['user_id']

        # Check if the user ID is already in the mapping
        if user_id in user_id_mapping:
            # If yes, replace the user ID with the corresponding number
            df.at[index, 'user_number'] = user_id_mapping[user_id]
        else:
            # If not, create a new number for the user ID and update the mapping
            user_id_mapping[user_id] = current_number
            df.at[index, 'user_number'] = current_number
            current_number += 1

    return df

# Example usage:
# Assuming you have a DataFrame 'df' with a column 'user_id'
# Replace it with your actual DataFrame and column names


df1 = replace_user_ids_with_numbers(df1)

print("Original DataFrame:")
df1.tail()

Original DataFrame:


,user_id,longitude,latitude,date_taken,photo/video_page_url,x,y,user_number
1876,99761031@N00,139.798172,35.631066,2014-03-22 15:59:58.0,http://www.flickr.com/photos/99761031@N00/1337...,1.556226e+07,4.249975e+06,1821.0
6773,9978960@N02,139.713134,35.684210,2007-07-21 12:36:05.0,http://www.flickr.com/photos/9978960@N02/11746...,1.555279e+07,4.257256e+06,1822.0
5718,99872288@N00,139.767111,35.668749,2009-06-23 06:31:37.0,http://www.flickr.com/photos/99872288@N00/3702...,1.555880e+07,4.255137e+06,1823.0
7760,9987404@N04,139.598500,35.569833,2013-05-19 10:36:57.0,http://www.flickr.com/photos/9987404@N04/87525...,1.554003e+07,4.241591e+06,1824.0
2342,99945529@N00,139.701202,35.658641,2007-11-27 13:24:17.0,http://www.flickr.com/photos/99945529@N00/2068...,1.555147e+07,4.253752e+06,1825.0


In [ ]:
# df1[df1['user_number'] == 300]

In [ ]:
def replace_item_ids_with_numbers(df):
    # Create a dictionary to map unique user IDs to numbers
    loc_id_mapping = {}
    current_number = 1  # Start numbering from 1

    # Iterate through the rows of the DataFrame
    for index, row in df.iterrows():
        coord = (row['latitude'], row['longitude'])

        # Check if the user ID is already in the mapping
        if coord in loc_id_mapping:
            # If yes, replace the user ID with the corresponding number
            df.at[index, 'location_id'] = loc_id_mapping[coord]
        else:
            # If not, create a new number for the user ID and update the mapping
            loc_id_mapping[coord] = current_number
            df.at[index, 'location_id'] = current_number
            current_number += 1

    return df

In [ ]:
df1 = replace_item_ids_with_numbers(df1)

In [ ]:
df1.head()

,user_id,longitude,latitude,date_taken,photo/video_page_url,x,y,user_number,location_id
151,10012675@N05,139.725666,35.711935,2012-05-31 16:21:11.0,http://www.flickr.com/photos/10012675@N05/8256...,1.555419e+07,4.261056e+06,1.0,1.0
3688,10012675@N05,139.714357,35.663999,2012-05-31 02:57:38.0,http://www.flickr.com/photos/10012675@N05/8365...,1.555293e+07,4.254486e+06,1.0,2.0
155,100623025@N08,139.570327,35.696197,2014-01-23 16:36:35.0,http://www.flickr.com/photos/100623025@N08/121...,1.553690e+07,4.258899e+06,2.0,3.0
1769,100623025@N08,139.771391,35.627727,2014-01-24 16:24:11.0,http://www.flickr.com/photos/100623025@N08/121...,1.555928e+07,4.249517e+06,2.0,4.0
860,10070485@N08,139.776638,35.717615,2009-05-21 14:09:50.0,http://www.flickr.com/photos/10070485@N08/3744...,1.555986e+07,4.261835e+06,3.0,5.0


In [ ]:
# # Count occurrences of each user ID
# user_counts = df1['user_id'].value_counts()

# # Filter user IDs with more than 5 occurrences
# user_ids_more_than_5_occurrences = user_counts[user_counts > 5].index.tolist()

In [ ]:
# Calculate Staying time

# num1 = df1['date_taken'][0]
# num2 = df1['date_taken'][1]
# num1
# num1 = num1.split()[1].split('.')[0:2][0]
# num2 = num2.split()[1].split('.')[0:2][0]
# print(num1, num2)

stayingtime = []
user = []
location = []

from datetime import datetime

def calculate_time_difference(df):

    for (index, current_row), (_, next_row) in zip(df.iterrows(), df.iloc[1:].iterrows()):

      if current_row['user_number'] == next_row['user_number'] and next_row['location_id'] == current_row['location_id']:
        num1 = current_row['date_taken']
        num2 = next_row['date_taken']
        time1_str = num1.split()[1].split('.')[0:2][0]
        time2_str = num2.split()[1].split('.')[0:2][0]

      # Convert string timestamps to datetime objects
        time_format = "%H:%M:%S"
        time1 = datetime.strptime(time1_str, time_format)
        time2 = datetime.strptime(time2_str, time_format)
        if time1 > time2:
          t2 = time1
          t1 = time2
      # Calculate the time difference
          time_difference = t2 - t1

        else:
          time_difference = time2 - time1

      # Extract hours, minutes, and seconds from the time difference
        hours, remainder = divmod(time_difference.seconds, 3600)
        minutes, seconds = divmod(remainder, 60)

      # Format the result as a string
        result = hours*60 + minutes
        stayingtime.append(result)
        user.append(df['user_number'][index])
        location.append(df['location_id'][index])




result = df1.groupby(['user_number', 'location_id']).filter(lambda x: len(x) > 1)
print(result)

In [ ]:
df1 = df1.sort_values(['user_number', 'location_id'])

In [ ]:
df1.head(20)

,user_id,longitude,latitude,date_taken,photo/video_page_url,x,y,user_number,location_id
151,10012675@N05,139.725666,35.711935,2012-05-31 16:21:11.0,http://www.flickr.com/photos/10012675@N05/8256...,1.555419e+07,4.261056e+06,1.0,1.0
3688,10012675@N05,139.714357,35.663999,2012-05-31 02:57:38.0,http://www.flickr.com/photos/10012675@N05/8365...,1.555293e+07,4.254486e+06,1.0,2.0
155,100623025@N08,139.570327,35.696197,2014-01-23 16:36:35.0,http://www.flickr.com/photos/100623025@N08/121...,1.553690e+07,4.258899e+06,2.0,3.0
1769,100623025@N08,139.771391,35.627727,2014-01-24 16:24:11.0,http://www.flickr.com/photos/100623025@N08/121...,1.555928e+07,4.249517e+06,2.0,4.0
860,10070485@N08,139.776638,35.717615,2009-05-21 14:09:50.0,http://www.flickr.com/photos/10070485@N08/3744...,1.555986e+07,4.261835e+06,3.0,5.0
4224,10070485@N08,139.769471,35.662151,2009-05-08 08:03:40.0,http://www.flickr.com/photos/10070485@N08/4114...,1.555907e+07,4.254233e+06,3.0,6.0
6141,10076455@N04,139.700775,35.657714,2008-09-14 05:16:58.0,http://www.flickr.com/photos/10076455@N04/2856...,1.555142e+07,4.253625e+06,4.0,7.0
3394,10076455@N04,139.766521,35.670022,2006-03-11 09:21:27.0,http://www.flickr.com/photos/10076455@N04/8105...,1.555874e+07,4.255311e+06,4.0,8.0
3965,10148151@N06,139.640483,35.602083,2010-11-24 16:13:06.0,http://www.flickr.com/photos/10148151@N06/5203...,1.554471e+07,4.246006e+06,5.0,9.0
982,10148151@N06,139.696676,35.667250,2012-05-13 12:25:45.0,http://www.flickr.com/photos/10148151@N06/7185...,1.555096e+07,4.254932e+06,5.0,10.0


In [ ]:
len(user)

2634

In [ ]:
calculate_time_difference(df1)

In [ ]:
print(stayingtime)
print(user)
print(location)

[1344, 11, 208, 1220, 241, 204, 1129, 167, 0, 194, 28, 1139, 1401, 1438, 1315, 1175, 788, 1377, 724, 0, 1439, 196, 1244, 1383, 9, 964, 1297, 275, 46, 1352, 1408, 759, 922, 529, 1031, 1265, 470, 1, 1438, 56, 135, 1297, 1127, 38, 223, 987, 1304, 1374, 1079, 1380, 15, 1418, 4, 1, 1431, 1290, 37, 198, 1219, 148, 1139, 209, 1073, 1361, 747, 891, 1269, 151, 1301, 199, 17, 1282, 518, 812, 57, 1314, 597, 130, 1400, 6, 349, 1417, 278, 149, 1309, 1418, 100, 40, 1007, 454, 1145, 16, 46, 1435, 1434, 1323, 325, 1240, 290, 1153, 25, 1401, 215, 27, 26, 1400, 1120, 318, 10, 1438, 876, 472, 92, 1427, 177, 1439, 250, 265, 39, 1431, 2, 418, 221, 297, 282, 982, 1118, 94, 1219, 330, 1432, 19, 18, 1402, 45, 1371, 1433, 1379, 59, 35, 92, 21, 411, 1433, 142, 1294, 818, 1301, 1421, 669, 1074, 1279, 1389, 647, 843, 1328, 1003, 553, 1276, 1439, 124, 1368, 175, 970, 98, 1419, 38, 1369, 1438, 454, 939, 808, 1165, 7, 924, 414, 35, 976, 580, 1079, 17, 1211, 1425, 17, 1419, 1357, 842, 133, 152, 73, 1435, 1406, 223, 1

In [ ]:
rating_df = pd.DataFrame()
rating_df['userId'] = user
rating_df['itemId'] = location
rating_df['staying_time'] = stayingtime

In [ ]:
mean_staytime = rating_df.groupby('itemId')['staying_time'].mean()
rating_df = rating_df.merge(mean_staytime, left_on='itemId', right_index=True, suffixes=('', '_mean'))
rating_df['normalized_staying_time'] = rating_df['staying_time'] / rating_df['staying_time_mean']
rating_df['rating'] =round((rating_df['normalized_staying_time'] - 0) / (5 - 0) * 5)

In [ ]:
rating_df.head()

,userId,itemId,staying_time,staying_time_mean,normalized_staying_time,rating
0,6.0,13.0,95,95.0,1.000000,1.0
1,6.0,14.0,11,11.0,1.000000,1.0
2,8.0,22.0,208,208.0,1.000000,1.0
3,11.0,27.0,219,228.2,0.959684,1.0
4,11.0,27.0,241,228.2,1.056091,1.0


In [ ]:
rating_df['userId'] = rating_df['userId'].astype(int)
rating_df['itemId'] = rating_df['itemId'].astype(int)
rating_df['rating'] = rating_df['rating'].astype(float)


### Collaborative Filtering

In [ ]:
# User Item Rating Matrix

userItemRatingMatrix=pd.pivot_table(rating_df, values='rating',
                                    index=['userId'], columns=['itemId'])
userItemRatingMatrix

itemId,13,14,22,27,32,34,45,47,48,49,...,7212,7213,7225,7235,7247,7248,7276,7277,7280,7282
userId,,,,,,,,,,,,,,,,,,,,,
6,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1798,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
1817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN


In [ ]:
def similarity(user1,user2):
    try:
        user1=np.array(user1)-np.nanmean(user1)
        user2=np.array(user2)-np.nanmean(user2)
        commonItemIds=[i for i in range(len(user1)) if user1[i]>0 and user2[i]>0]
        if len(commonItemIds)==0:
           return 0
        else:
           user1=np.array([user1[i] for i in commonItemIds])
           user2=np.array([user2[i] for i in commonItemIds])
           return correlation(user1,user2)
    except ZeroDivisionError:
        print("You can't divide by zero!")

In [ ]:
def nearestNeighbourRatings(activeUser,K):
    try:
        similarityMatrix=pd.DataFrame(index=userItemRatingMatrix.index,columns=['Similarity'])

        for i in userItemRatingMatrix.index:
            similarityMatrix.loc[i]=similarity(userItemRatingMatrix.loc[activeUser],userItemRatingMatrix.loc[i])

        similarityMatrix=pd.DataFrame.sort_values(similarityMatrix,['Similarity'],ascending=[0])
        nearestNeighbours=similarityMatrix[:K]
        neighbourItemRatings=userItemRatingMatrix.loc[nearestNeighbours.index]
        predictItemRating=pd.DataFrame(index=userItemRatingMatrix.columns, columns=['Rating'])

        for i in userItemRatingMatrix.columns:
            predictedRating=np.nanmean(userItemRatingMatrix.loc[activeUser])
            for j in neighbourItemRatings.index:
                if userItemRatingMatrix.loc[j,i]>0:
                   predictedRating += (userItemRatingMatrix.loc[j,i]-np.nanmean(userItemRatingMatrix.loc[j]))*nearestNeighbours.loc[j,'Similarity']

                predictItemRating.loc[i,'Rating']=predictedRating
    except ZeroDivisionError:
        print("You can't divide by zero!")
    return predictItemRating

In [ ]:
def topNRecommendations(activeUser,N):
    try:
        predictItemRating=nearestNeighbourRatings(activeUser,10)
        placeAlreadyWatched=list(userItemRatingMatrix.loc[activeUser]
                              .loc[userItemRatingMatrix.loc[activeUser]>0].index)
        predictItemRating=predictItemRating.drop(placeAlreadyWatched)
        topRecommendations=pd.DataFrame.sort_values(predictItemRating,
                                                ['Rating'],ascending=[0])[:N]
        # topRecommendationTitles=(placeInfo.loc[placeInfo.itemId.isin(topRecommendations.index)])
    except ZeroDivisionError:
        print("You can't divide by zero!")
    return list(topRecommendations.index)

In [ ]:
activeUser=float(input("Enter userid: "))
print("The recommended places for you are: ")
print(topNRecommendations(activeUser,5))

Enter userid: 8
The recommended places for you are: 
[13, 4512, 4440, 4441, 4443]
